In [1]:
#importing libraries
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
spark = SparkSession.builder \
    .appName("RealTimeFraudDetection") \
    .getOrCreate()


25/04/17 18:21:38 WARN Utils: Your hostname, Dulyas-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.177.129 instead (on interface en0)
25/04/17 18:21:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/17 18:21:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/17 18:21:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
pandas_df1 = pd.read_csv('/Users/dulyanethmini/Desktop/Y 4 S1/BDP/BDP Group project/coding/fraud detection.csv', nrows=10000) 


In [7]:
df1 = spark.createDataFrame(pandas_df1)

In [9]:
df1.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [17]:
from pyspark.ml.feature import StringIndexer

# Drop the column if it already exists
if 'type_indexed' in df1.columns:
    df1 = df1.drop('type_indexed')

# Apply StringIndexer
indexer = StringIndexer(inputCol="type", outputCol="type_indexed")
df1 = indexer.fit(df1).transform(df1)


In [27]:

# Select relevant features
feature_cols = ["type_indexed", "amount", "oldbalanceOrg", "newbalanceOrig", 
                "oldbalanceDest", "newbalanceDest"]

# Vectorize the features
vectorAssembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df1 = vectorAssembler.transform(df1)

# Keep only required columns
model_data = df1.select("features", col("isFraud").alias("label"))



IllegalArgumentException: Output column features already exists.

In [21]:
df1.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_indexed|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|         0.0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|         0.0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|         3.0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|      

In [29]:
df1 = df1.drop("features")

# Now transform again
df1 = vectorAssembler.transform(df1)


In [31]:
# Keep only required columns
model_data = df1.select("features", col("isFraud").alias("label"))

In [35]:
print(feature_cols)


['type_indexed', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']


In [37]:
train_data, test_data = model_data.randomSplit([0.8, 0.2], seed=42)


In [39]:
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100)
rf_model = rf.fit(train_data)


In [41]:
predictions = rf_model.transform(test_data)
predictions.select("label", "prediction", "probability").show(10, truncate=False)

evaluator = BinaryClassificationEvaluator()
auc = evaluator.evaluate(predictions)
print(f"AUC Score: {auc:.4f}")


+-----+----------+------------------------------------------+
|label|prediction|probability                               |
+-----+----------+------------------------------------------+
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9986256468535711,0.0013743531464288196]|
|0    |0.0       |[0.9976007936451956,0.0023992063548044294]|
|0    |0.0       |[0.9976007936451956,0.0023992063548044294]|
+-----+----------+------------------------------------------+
only showing top 10 rows

AUC Score: 0.9974


In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Run prediction on test set
predictions = rf_model.transform(test_data)

# Initialize evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",  # or whatever your label column is called
    predictionCol="prediction",
    metricName="accuracy"
)

# Compute accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy:.4f}")


Model Accuracy: 0.9959
